<a href="https://colab.research.google.com/github/KokiNiimura/study/blob/master/val_psp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/study/PyTorch_Advanced/03

/content/drive/My Drive/study/PyTorch_Advanced/03


In [2]:
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [4]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", 
                           transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", 
                           transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

batch_size = 4

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [5]:
from utils.pspnet import PSPNet

net = PSPNet(n_classes=21)
net.load_state_dict(torch.load("./weights/pspnet50_30.pth"))

<All keys matched successfully>

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net.to(device)

cuda:0


PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

In [21]:
def val_model(net, dataloaders_dict, criterion):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device: {}".format(device))

    net.to(device)

    torch.backends.cudnn.benchmark = True

    num_val_imgs = len(dataloaders_dict["val"].dataset)

    net.eval()

    val_loss = 0.0
    with torch.no_grad():
        for images, anno_class_images in dataloaders_dict["val"]:
            if images.size()[0] == 1:
                continue
            
            images = images.to(device)
            anno_class_images.to(device)

            outputs = net(images)
            loss = criterion(outputs[0], anno_class_images.long().to(device))

            val_loss += loss.item() 

    val_loss /= num_val_imgs
    
    return val_loss

In [22]:
loss = val_model(net, dataloaders_dict, criterion)

device: cuda:0


In [23]:
loss

0.2559994383463784